In [1]:
import datetime as dt
import time
import logging
from IPython.display import clear_output

from optibook.synchronous_client import Exchange

from math import floor, ceil
from black_scholes import call_value, put_value, call_delta, put_delta
from libs import calculate_current_time_to_date

e = Exchange()
e.connect()

logging.getLogger('client').setLevel('INFO')

2022-03-30 21:38:12,053 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2022-03-30 21:38:12,060 [client    ] [Thread-4    ] background thread started
2022-03-30 21:38:12,080 [client    ] [Thread-4    ] opened connection
2022-03-30 21:38:12,081 [client    ] [Thread-4    ] start read <StreamReader t=<_SelectorSocketTransport fd=91 read=polling write=<idle, bufsize=0>>>
2022-03-30 21:38:12,090 [client    ] [Thread-4    ] logged in!


In [2]:
def get_order_book_for_instrument(instrument_id):
    """
    Fetching bid and ask books.
    """
    
    book = e.get_last_price_book(instrument_id)
    
    return book.bids, book.asks

In [3]:
def get_positions(instrument_id):
    """
    Given an instrument id returns the position for this instrument.
    """
    position = e.get_positions()[instrument_id]
    return position

In [4]:
def adjust_position(position, instrument_id, bids, asks):
    """
    Takes in a position, an instrument id and bid and ask books.
    Inserts aggressive IOC orders to adjust the position of the instrument, 
    either at the ask or bid side depending on the position.
    Returns the adjusted position.
    """
    if position > 0:
        e.insert_order(instrument_id=instrument_id, price=bids[0].price, volume=10, side='ask', order_type='ioc')
    if position < 0:
        e.insert_order(instrument_id=instrument_id, price=asks[0].price, volume=10, side='bid', order_type='ioc')
    position = e.get_positions()[instrument_id]
    print(f'Adjusting position for {instrument_id}')
    return position

In [5]:
def delete_all_outstanding_orders_for_option(instrument_id):
    """
    Deletes alll outstanding orders given an instrument_id
    """
    orders = e.get_outstanding_orders(instrument_id)
    for order_id in orders:
        e.delete_order(instrument_id, order_id=order_id)

In [6]:
def calculate_fair_bid_price(option, bid, ask):
    """
    Takes in the option dictionary and bid and ask prices.
    Based on whether it is a call or a put option in calculates the fair bid price
    the call or put Black-Scholes formula is used.
    Return the fair bid price.
    """
    # Calculate time to expiry
    time_to_expiry = calculate_current_time_to_date(option['expiry_date'])
    # Distinguish between call and put in order the use the right function
    if option['callput'] == 'call':
        fair_bid_price = call_value(S = bid, K = option['strike'], T = time_to_expiry, r = r, sigma = sigma)
    elif option['callput'] == 'put':
        fair_bid_price = put_value(S = ask, K = option['strike'], T = time_to_expiry, r =r, sigma = sigma)
    return fair_bid_price

In [7]:
def calculate_fair_ask_price(option, bid, ask):
    """
    Takes in the option dictionary and bid and ask prices.
    Based on whether it is a call or a put option in calculates the fair ask price
    the call or put Black-Scholes formula is used.
    Return the fair ask price.
    """
    # Calculate time to expiry
    time_to_expiry = calculate_current_time_to_date(option['expiry_date'])
    # Distinguish between call and put in order the use the right function
    if option['callput'] == 'call':
        fair_ask_price = call_value(S = ask, K = option['strike'], T = time_to_expiry, r = r, sigma = sigma)
    elif option['callput'] == 'put':
        fair_ask_price = put_value(S = bid, K = option['strike'], T = time_to_expiry, r =r, sigma = sigma)
    return fair_ask_price

In [8]:
def desired_bid_price(fair_bid_price, offset, tick):
    """
    Takes in fair bid price, offset and the tick.
    Substracts the offset from the fair bid price and rounds down while adjusting to the tick size.
    Return the desired bid price.
    """
    desired_bid_price = floor((fair_bid_price - offset) / tick) * tick
    return desired_bid_price

def desired_ask_price(fair_ask_price, offset, tick):
    """
    Takes in fair ask price, offset and the tick.
    Adds the offset to the fair ask price and rounds up while adjusting to the tick size.
    Return the desired ask price.
    """
    desired_ask_price = ceil((fair_ask_price + offset) / tick) * tick
    return desired_ask_price

In [9]:

def market_making_bid(option, desired_bid_price, volume):
    """
    Takes in the option dictionary, desired bid price and desired volume.
    Inserts limit orders on for the desired bid price for a given a volume
    """
    print(f'''Inserting bid for {option['id']}: {volume:.0f} lot(s) at price {desired_bid_price:.2f}.''')
    e.insert_order(instrument_id=option['id'], price=desired_bid_price, volume=volume, side='bid', order_type='limit')


def market_making_ask(option, desired_ask_price, volume):
    """
    Takes in the option dictionary, desired ask price and desired volume.
    Insert limit orders on for the desired ask price for a given a volume
    """
    print(f'''Inserting ask for {option['id']}: {volume:.0f} lot(s) at price {desired_ask_price:.2f}.''')
    e.insert_order(instrument_id=option['id'], price=desired_ask_price, volume=volume, side='ask', order_type='limit')


def market_making(option, desired_bid_price, desired_ask_price, volume=15):
    """
    Takes in Takes in the option dictionary, desired bid price, desired ask price and desired volume.
    Makes a market by combining the functions for insterting bids and asks prices
    """
    market_making_bid(option, desired_bid_price, volume)
    market_making_ask(option, desired_ask_price, volume)

In [10]:
def market_making_for_options(OPTIONS):
    """
    Takes in a list of dictionaries of options.
    Loops through the available options and makes a market based on calculation of the prices using the Black-Scholes formula.
    Returns the order books for the underlying used to calculate the fair prices.
    """
    BMW_book_bids, BMW_book_asks = get_order_book_for_instrument('BMW')
    # Check if books are not empty
    if BMW_book_bids and BMW_book_asks:
        print(f'The observed best bid and ask for BMW are {BMW_book_bids[0].price:.2f} and {BMW_book_asks[0].price:.2f}')
        
        # Loop through all the options to make the market
        for option in OPTIONS:
            clear_output(wait=True)
            
            # Step 1: Delete all existing outstanding orders
            delete_all_outstanding_orders_for_option(option['id'])
            
            # Get the current prices for the options
            current_bids, current_asks = get_order_book_for_instrument(option['id'])
            print(f'The current option bid and ask for {option["id"]} are {current_bids[0].price:.2f} and {current_asks[0].price:.2f}')
            
            # Step 0: Get position and adjust if it is close to the limit 
            position = get_positions(option['id'])
            
            position = adjust_position_within_limits(position, option['id'], 80, current_bids, current_asks)
            while abs(position) > 80:
                clear_output(wait=True)
                position = adjust_position(position, option['id'], current_bids, current_asks)
                current_bids, current_asks = get_order_book_for_instrument(option['id'])
                time.sleep(0.1)

            # Step 2: Calculate the fair bid and ask prices
            fair_bid_price = calculate_fair_bid_price(option, BMW_book_bids[0].price, BMW_book_asks[0].price)
            print(f'The fair bid price for {option["id"]} is {fair_bid_price:.2f}')
            fair_ask_price = calculate_fair_ask_price(option, BMW_book_bids[0].price, BMW_book_asks[0].price)
            print(f'The fair ask price for {option["id"]} is {fair_ask_price:.2f}')

            # Step 3: Calculate the desired prices
            current_desired_bid_price = desired_bid_price(fair_bid_price, 0, tick)
            print(f'The desired bid price for {option["id"]} is {current_desired_bid_price:.2f}')
            current_desired_ask_price = desired_ask_price(fair_ask_price, 0, tick)
            print(f'The desired ask price for {option["id"]} is {current_desired_ask_price:.2f}')
            
            # Step 4: Insert limit orders on the desired bid and ask price with a volume of 5
            market_making(option, current_desired_bid_price, current_desired_ask_price, volume=5)
            time.sleep(0.1)
    return BMW_book_bids, BMW_book_asks

In [11]:
def adjust_position_within_limits(position, instrument_id, threshold, bids, asks):       
    """
    Takes in position, instrument id, threshold, bids orderbook, asks orderbook.
    Adjust the position of the given instrument untill its absolute value is beneath the given threshold.
    Continuously updates the orderbooks to get the best prices.
    Returns the adjusted position.
    """
    while abs(position) > threshold:
        clear_output(wait=True)
        position = adjust_position(position, instrument_id, bids, asks)
        bids, asks = get_order_book_for_instrument(instrument_id)
        time.sleep(0.1)
    return position

In [12]:
def calculate_mid_point(bids, asks, tick):
    """
    Takes in bids and asks order books and the tick size.
    Returns midpoint between best bid and ask
    """
    return int(((bids[0].price + asks[0].price) / 2) / tick) * tick

In [13]:
def calculate_delta_for_option(option, observed_price):
    """
    Takes in dictionary for option and observerd price.
    Depending on whether the option is a call or put option calculates the delta
    using the correct function based on the Black-Scholes formula.
    Return the delta
    """
    time_to_expiry = calculate_current_time_to_date(option['expiry_date'])
    if option['callput'] == 'call':
        delta = call_delta(S = observed_price, K = option['strike'], T = time_to_expiry, r = r, sigma = sigma)
    elif option['callput'] == 'put':
        delta = put_delta(S = observed_price, K = option['strike'], T = time_to_expiry, r = r, sigma = sigma)
    return delta

In [14]:
def total_delta_for_options(bids, asks):
    """
    Takes in the bids and asks order books.
    Calculates the total delta for the options by looping through the options
    and calculating the product of the position of the option and the delta for the option.
    These are added up to get the total delta for the options.
    Returns the total delta for the options
    """
    total_delta_for_options = 0
    mid_point = calculate_mid_point(bids, asks, tick)
    # loop through options
    for option in OPTIONS:
        # Get current positions for this option
        option_position = e.get_positions()[option['id']]

        # Calculate delta using the same inputs as for the previously calculated fair price we calculated but now using the midpoint
        delta_for_option = calculate_delta_for_option(option, mid_point)      
        print(f'Delta for {option["id"]} is {delta_for_option}')
        
        # Calculate outstanding delta for this option
        outstanding_delta = delta_for_option * option_position
        print(f'The outstanding delta for {option["id"]} is {outstanding_delta}')
        
        total_delta_for_options += outstanding_delta      
    print(f'The total outstanding delta for the options is {total_delta_for_options}')
    return total_delta_for_options

In [15]:
# Calculate how much stock of BMW we need to buy/sell to be minimize our delta again
def calculate_volume_to_hedge(total_delta_for_options, BMW_position):
    """
    Takes in total delta for options and the position of the underlying.
    Calculates the volume to hedge total position as the sum of the total delta for options
    and the position of the underlying.
    Returns the volume the hedge
    """
    volume_to_hedge = total_delta_for_options + BMW_position
    return volume_to_hedge

In [16]:
def calculate_allowed_volume(position):
    """
    Takes in position.
    Calculates the allowed volume for an order given the position.
    Returns allowed volume.
    """
    allowed_volume = 100-abs(position)
    return allowed_volume

In [17]:
# Calulate the volume we try to trade with while staying in the position limit of the BMW stock
def calculate_trade_volume(volume_to_hedge, allowed_volume):
    """
    Takes in volume to hedge and allowed volume.
    Calculates the trade volume as the minimum of the absolute value of the volume to hedge
    and the allowed volume making sure it is an integer value.
    Return the trade volume.   
    """
    trade_volume = int(min(abs(volume_to_hedge), allowed_volume))
    return trade_volume

In [18]:
def negative_volume_hedge_order(order_book, volume):
    """
    Takes in an order book and a volume.
    Insert an IOC order for the underlying stock at the bid side for the best price in the order book for the given volume.
    """
    print(f'''Inserting bid for BMW: {volume:.0f} lot(s) at price {order_book[0].price:.2f}.''')
    e.insert_order(instrument_id='BMW', price=order_book[0].price, volume=volume, side='bid', order_type='ioc')

In [19]:
def positive_volume_hedge_order(order_book, volume):
    """
    Takes in an order book and a volume.
    Insert an IOC order for the underlying stock at the ask side for the best price in the order book for the given volume.
    """
    print(f'''Inserting ask for BMW: {volume:.0f} lot(s) at price {order_book[0].price:.2f}.''')
    e.insert_order(instrument_id='BMW', price=order_book[0].price, volume=volume, side='ask', order_type='ioc')

In [20]:
def adjust_volume_to_hedge(bids, asks):
    """
    Takes in bids and asks order books.
    Given the current position for the underlying calculates the adjusted volume to hedge.
    Return the adjusted volume to hedge.
    """
    BMW_position = e.get_positions()['BMW']
    total_delta_for_the_options = total_delta_for_options(bids, asks)
    adjusted_volume_to_hedge = calculate_volume_to_hedge(total_delta_for_the_options, BMW_position)
    return adjusted_volume_to_hedge

In [21]:
def hedge_position(volume_to_hedge, current_book_bids, current_book_asks, old_book_bids, old_book_asks, trade_volume):
    """
    Takes in volume to hedge, bids and asks order books, trade volume.
    If the volume to hedge is positive it calls a function that inserts a IOC order at the bid side
    for the best price in the order book for trade volume such that we sell the underlying stock.
    If the volume to hedge is negative it calls a function that inserts a IOC order at the ask side
    for the best price in the order book for trade volume such that we buy the underlying stock.
    After this the volume to hedge is adjusted.
    Returns volume to hedge.
    """
    if volume_to_hedge < 0:
        negative_volume_hedge_order(current_book_asks, trade_volume)
    elif volume_to_hedge > 0:
        positive_volume_hedge_order(current_book_bids, trade_volume)
    # Use the same bids and asks prices used to determine the fair prices to recalculate the volume
    volume_to_hedge = adjust_volume_to_hedge(old_book_bids, old_book_asks)
    print(f'The volume to hedge is {volume_to_hedge}')
    return volume_to_hedge

In [22]:
def hedge_the_volume(volume_to_hedge, old_book_bids, old_book_asks, trade_volume, threshold = 15):
    """
    Takes in volume to hedge, the bids and asks books used to calculate the fair price and the trade volume.
    Optionally a threshold can be given in, the default is 15.
    While the volume to hedge is above threshold it adjust this position by trading the stocks with an
    aggressive IOC against the current top level of the stock. 
    The volume to hedge, position of the underlying, allowed volume and trade volume are adjusted
    """
    while abs(volume_to_hedge) > threshold:
        clear_output(wait=True)
        current_BMW_book_bids, current_BMW_book_asks = get_order_book_for_instrument('BMW')
        if BMW_book_bids and BMW_book_asks:
            print(f'The volume to hedge is {volume_to_hedge}')
            volume_to_hedge = hedge_position(volume_to_hedge, current_BMW_book_bids, current_BMW_book_asks, old_book_bids, old_book_asks, trade_volume) #could adjust price to be more certain
            print(f'The adjusted volume to hedge is {volume_to_hedge}')
            BMW_position = get_positions('BMW')
            allowed_volume = calculate_allowed_volume(BMW_position)
            trade_volume = calculate_trade_volume(volume_to_hedge, allowed_volume)
            print(f'The adjusted trade volume is {trade_volume}')
            time.sleep(0.1)

In [23]:
def delta_hedge(BMW_book_bids, BMW_book_asks):
    """
    Takes in the order books for the underlying used to calculate the fair prices.
    Step 0: First makes sure the position of the underlying is within limits.
    Step 1: Calculate the total outstanding delta for the options
    Step 2: Calculate the volume to hedge the delta (taking into account the position of the underlying).
    Step 3: While our delta is above certain threshold hedge it by inserting IOC orders.
    """
    
    # Save the input bids and asks
    bids = BMW_book_bids
    asks = BMW_book_asks
    
    # Get position for the underlying
    BMW_position = get_positions('BMW')

    # If we are at the limit position for the BMW stock we try to adjust this agressively
    BMW_position = adjust_position_within_limits(BMW_position,'BMW', 95, bids, asks)       
    print(f'The current BMW position is {BMW_position}.')   

    # Step 1
    # Calculate the total outstanding delta for the options using 
    # by using the Black-Scholes formulae
    total_delta_for_the_options = total_delta_for_options(BMW_book_bids, BMW_book_asks)
    
    # Include the stock deltas we hold as well
    volume_to_hedge = calculate_volume_to_hedge(total_delta_for_the_options, BMW_position)
    print(f'The volume to hedge is {volume_to_hedge}')

    # Make sure we stay within the total position for BMW, i.e. below 100
    allowed_volume = calculate_allowed_volume(BMW_position)
    print(f'The allowed volume is {allowed_volume}')

    # Step 2
    # Calculate the amount of stocks we should buy or sell based on our current position to
    # hedge the deltas
    trade_volume = calculate_trade_volume(volume_to_hedge, allowed_volume)
    print(f'The trade volume is {trade_volume}')

    # Step 3
    # Our position to hedge can not become more than 25, to stay on the safe
    # side we adjust it when it becomes higher than 15
    # by trading these stocks with an aggressive IOC against the current
    # top level of the stock
    hedge_the_volume(volume_to_hedge, bids, asks, trade_volume)

In [25]:
def printPositions():
    """
    For use in final 
    """
    positions = e.get_positions()
    for p in positions:
        print(p, positions[p])

In [26]:
def clear_all_positions():
    """
    Clear all positions regardless of losses incurred
    """
    print(e.get_positions())
    for s, p in e.get_positions().items():
        if p > 0:
            e.insert_order(s, price=1, volume=p, side="ask", order_type="ioc")
        elif p < 0:
            e.insert_order(s, price=100000, volume=-p, side="bid", order_type="ioc")  
    print(e.get_positions())

In [30]:
e = Exchange()
e.connect()

logging.getLogger('client').setLevel('ERROR')

STOCK_ID = 'BMW'
OPTIONS = [
    {'id': 'BMW-2022_04_29-050C', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 50, 'callput': 'call'},
    {'id': 'BMW-2022_04_29-050P', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 50, 'callput': 'put'},
    {'id': 'BMW-2022_04_29-075C', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 75, 'callput': 'call'},
    {'id': 'BMW-2022_04_29-075P', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 75, 'callput': 'put'},
    {'id': 'BMW-2022_04_29-100C', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 100, 'callput': 'call'},
    {'id': 'BMW-2022_04_29-100P', 'expiry_date': dt.datetime(2022, 4, 29, 12, 0, 0), 'strike': 100, 'callput': 'put'},
    {'id': 'BMW-2022_05_27-050C', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 50, 'callput': 'call'},
    {'id': 'BMW-2022_05_27-050P', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 50, 'callput': 'put'},
    {'id': 'BMW-2022_05_27-075C', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 75, 'callput': 'call'},
    {'id': 'BMW-2022_05_27-075P', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 75, 'callput': 'put'},
    {'id': 'BMW-2022_05_27-100C', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 100, 'callput': 'call'},
    {'id': 'BMW-2022_05_27-100P', 'expiry_date': dt.datetime(2022, 5, 27, 12, 0, 0), 'strike': 100, 'callput': 'put'}
]

tick = 0.1
sigma = 3.0
r = 0.0
clear_all_positions()

while True:
    clear_output(wait=True)
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')
    print(f'')
    # Perform the market making part of the algorithm
    BMW_book_bids, BMW_book_asks = market_making_for_options(OPTIONS)
    time.sleep(1)
    
    # If we are close to the limit position for the BMW stock we try to adjust this agressively
    BMW_position = get_positions('BMW')
    BMW_position = adjust_position_within_limits(BMW_position,'BMW', 95, BMW_book_bids, BMW_book_asks)     
    print(f'The current BMW position is {BMW_position}.')
    
    # Perform the delta hedging part of the algorithm
    delta_hedge(BMW_book_bids, BMW_book_asks)
    printPositions()
    time.sleep(1)

The current option bid and ask for BMW-2022_05_27-100P are 68.00 and 68.20
The fair bid price for BMW-2022_05_27-100P is 68.13
The fair ask price for BMW-2022_05_27-100P is 68.35
The desired bid price for BMW-2022_05_27-100P is 68.10
The desired ask price for BMW-2022_05_27-100P is 68.40
Inserting bid for BMW-2022_05_27-100P: 5 lot(s) at price 68.10.
Inserting ask for BMW-2022_05_27-100P: 5 lot(s) at price 68.40.


KeyboardInterrupt: 